In [2]:
#-*- coding: utf-8 -*-
"""
Created on Mon Sep 21 10:18:43 2015

@author: jml
"""

# ANOVA

import numpy
import pandas
import statsmodels.formula.api as smf 
import statsmodels.stats.multicomp as multi
import seaborn
import matplotlib.pyplot as plt


data = pandas.read_csv('diet_exercise.csv', low_memory=False)

#data["Diet"] = data["Diet"].astype('category')
#data['WeightLoss']= data['WeightLoss'].convert_objects(convert_numeric=True)

#dir(smf)

model1 = smf.ols(formula='WeightLoss ~ C(Diet)', data=data).fit()
print (model1.summary())

sub1 = data[['WeightLoss', 'Diet']].dropna()

print ("means for WeightLoss by Diet A vs. B")
m1= sub1.groupby('Diet').mean()
print (m1)

print ("standard deviation for mean WeightLoss by Diet A vs. B")
st1= sub1.groupby('Diet').std()
print (st1)

# bivariate bar graph
seaborn.factorplot(x="Diet", y="WeightLoss", data=data, kind="bar", ci=None)
plt.xlabel('Diet Type')
plt.ylabel('Mean Weight Loss in pounds')


sub2=data[(data['Exercise']=='Cardio')]
sub3=data[(data['Exercise']=='Weights')]

print ('association between diet and weight loss for those using Cardio exercise')
model2 = smf.ols(formula='WeightLoss ~ C(Diet)', data=sub2).fit()
print (model2.summary())

print ('association between diet and weight loss for those using Weights exercise')
model3 = smf.ols(formula='WeightLoss ~ C(Diet)', data=sub3).fit()
print (model3.summary())

print ("means for WeightLoss by Diet A vs. B  for CARDIO")
m3= sub2.groupby('Diet').mean()
print (m3)
print ("Means for WeightLoss by Diet A vs. B for WEIGHTS")
m4 = sub3.groupby('Diet').mean()
print (m4)

# End of Lesson 2
#%%
# Beginning of Lesson 3

#subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1=data[(data['AGE']>=18) & (data['AGE']<=25) & (data['CHECK321']==1)]

#SETTING MISSING DATA
sub1['S3AQ3B1']=sub1['S3AQ3B1'].replace(9, numpy.nan)
sub1['S3AQ3C1']=sub1['S3AQ3C1'].replace(99, numpy.nan)

#recoding number of days smoked in the past month
recode1 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub1['USFREQMO']= sub1['S3AQ3B1'].map(recode1)

#converting new variable USFREQMMO to numeric
sub1['USFREQMO']= sub1['USFREQMO'].convert_objects(convert_numeric=True)

# Creating a secondary variable multiplying the days smoked/month and the number of cig/per day
sub1['NUMCIGMO_EST']=sub1['USFREQMO'] * sub1['S3AQ3C1']

#sub1['NUMCIGMO_EST']= sub1['NUMCIGMO_EST'].convert_objects(convert_numeric=True)

ct1= sub1.groupby('NUMCIGMO_EST').size()
print (ct1)

model1 = smf.ols(formula='NUMCIGMO_EST ~ MAJORDEPLIFE', data=sub1).fit()
print (model1.summary())

sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()

print ('means for numcigmo_est by major depression status')
m1= sub2.groupby('MAJORDEPLIFE').mean()
print (m1)

print ('standard deviations for numcigmo_est by major depression status')
sd1 = sub2.groupby('MAJORDEPLIFE').std()
print (sd1)

sub3 = sub1[['NUMCIGMO_EST', 'ETHRACE2A']].dropna()

model1 = smf.ols(formula='NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub3).fit()
print (model1.summary())

print ('means for numcigmo_est by major depression status')
m2= sub3.groupby('ETHRACE2A').mean()
print (m2)

print ('standard deviations for numcigmo_est by major depression status')
sd2 = sub3.groupby('ETHRACE2A').std()
print (sd2)

mc2 = multi.MultiComparison(sub3['NUMCIGMO_EST'], sub3['ETHRACE2A'])
res2 = mc2.tukeyhsd()
print(res2.summary())

OSError: File b'diet_exercise.csv' does not exist